### Demonstration of Stern-Gerlach simulator of a Harmonic Oscillator System with alternating measurements of position and energy

In [3]:
### Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import math
from numpy.polynomial.hermite import *

### Parameters for the initial state, which will
### be a Gaussian wavepacket centered
### at x0 with spread given by sig
### and average momentum given by k0
### all values in atomic units
x0 = 1.0
sig = 0.5
k0 = 0.5

### More constants
# hbar in atomic units
hbar = 1
# pi
pi = np.pi
# force constant in atomic units
k = 1.
# reduced mass in atomic units
mu = 300.

### Establish x array and initialize plot objects
x = np.linspace(-5,5,200)
fig, ax = plt.subplots()
plt.close()

ax.set_xlim(( -5, 5))
ax.set_ylim((-2, 2))

line, = ax.plot([], [], lw=2)
pot,  = ax.plot([], [], lw=2)

def init():
    line.set_data([], [])
    pot.set_data([], [])
    return (line, pot,)

# function that returns an array containing the Gaussian 
# wavepacket values evaluated along x
def gaussian_packet(x, x0, sig, k0):
    ci = 0+1j
    T1 = 1/(sig * np.sqrt(2 * np.pi))
    T2 = np.exp(-0.5 * ((x-x0)/sig)**2)
    T3 = np.exp(ci * k0 * x)
    return T1 * T2 * T3

# function that returns the harmonic potential 
# evaluated along x
def harmonic_potential(k, x):
    return 0.5 * k * x ** 2

# function that returns a model of a position 
# eigenfunction, which is just a narrow gaussian function
# with avg momentum equal to zero
def position_eigenfunction(x, x0, sig):
    ci = 0+1j
    T1 = 1/(sig * np.sqrt(2 * np.pi))
    T2 = np.exp(-0.5 * ((x-x0)/sig)**2)
    return T1 * T2 

#
def energy_eigenfunction(state, x, k, mu):
    w = np.sqrt(k/mu)
    psi = np.zeros_like(x)
    herm_coeff = []
    
    # the numpy hermite polynomial function hermval takes
    # an array of coefficients specifying the weight of the 
    # order of the polynomial you want... since we want 
    # only the hermite polynomial of order n corresponding to
    # the quantum number, we want this to be an array of
    # zeros except for the entry corresponding to the order n, which
    # will have the value 1
    for i in range(state):
        herm_coeff.append(0)
    herm_coeff.append(1)
    
    for i in range(0,len(x)): # in xgrid:
        psi[i] = math.exp(-mu*w*x[i]**2/(2*hbar)) * hermval((mu*w/hbar)**0.5 * x[i], herm_coeff)
        
    psi = np.multiply(psi, 1 / (math.pow(2, state) * math.factorial(state))**0.5 * (mu*w/(pi*hbar))**0.25)
    
    return psi

def energy_eigenvalue(n,k,mu):
    return np.sqrt(k/mu) * (n+(1./2))

def time_component(k, mu, n, t):
    ci = 0+1j
    En = energy_eigenvalue(n, k, mu)
    return np.exp(-ci*En*t) 


# initialize wavefunction as a gaussian wavepacket
Psi_gp = gaussian_packet(x, x0, sig, k0)

# array of quantum numbers for the expansion of 
# the wavefunction in terms of energy eigenfunctions
n_array = np.linspace(0,50,51)
# array of expansion coefficients
cn_array = np.zeros(len(n_array),dtype=complex)


y_exp = np.zeros_like(Psi_gp)
for i in range(0,51):
    psi = energy_eigenfunction(int(n_array[i]), x, k, mu)
    integrand = np.conj(psi)*Psi_gp
    cn_array[i] = np.trapz(integrand, x)
    y_exp = y_exp + cn_array[i] * psi
    
N_time = 200
n0 = np.zeros(1)

print(n_array)
def animate(i):
    y = np.zeros(len(x),dtype=complex)
    p_of_x = harmonic_potential(k, x)
    
    if i<50:
        for j in range(0,51):
            ft = time_component(k, mu, int(n_array[j]), i)
            fx = energy_eigenfunction(int(n_array[j]), x, k, mu)
            y = y + cn_array[j] * fx * ft

    elif i==50:
        for j in range(0,51):
            ft = time_component(k, mu, int(n_array[j]), (i-50))
            fx = energy_eigenfunction(int(n_array[j]), x, k, mu)
            y = y + cn_array[j] * fx * ft

        P = np.real(np.conj(y) * y)

        norm = np.sum(P)
        P_norm = P / norm

        p0 = np.random.choice(x, 1, p=P_norm)
        print(" Position measured to be at ",p0)
        pf = position_eigenfunction(x, p0[0], 0.2)
        y = pf

        for j in range(0,51):
            psi = energy_eigenfunction(int(n_array[j]), x, k, mu)
            integrand = np.conj(psi)*pf
            cn_array[j] = np.trapz(integrand, x)
            ft = time_component(k, mu, int(n_array[j]), (i-50))
            y = y + cn_array[j] * psi

    elif i<150:
        for j in range (0,51):
            ft = time_component(k, mu, int(n_array[j]), (i-50))
            fx = energy_eigenfunction(int(n_array[j]), x, k, mu)
            y = y + cn_array[j] * fx * ft  

    elif i==150:
        pn = np.real(np.conj(cn_array) * cn_array)
        norm = np.sum(pn)
        pn_norm = pn/norm
        nval = np.random.choice(n_array, 1, p=pn_norm)
        n0[0] = nval[0]
        En0 = energy_eigenvalue(n0[0], k, mu)
        print(" Randomly measured state", n0, "which has energy ",En0)
        ft = time_component(k, mu, int(n0[0]), (i-150))
        fx = energy_eigenfunction(int(n0[0]), x, k, mu)
        y = fx * ft

    else:
        ft = time_component(k, mu, int(n0[0]), i)
        fx = energy_eigenfunction(int(n0[0]), x, k, mu)
        y = fx * ft 
   
    line.set_data(x, np.real(y))
    pot.set_data(x, p_of_x)
    
    return (line, pot,)
   
anim = animation.FuncAnimation(fig, animate, init_func=init,
                        frames=N_time, interval=100, blit=True)

rc('animation', html='jshtml')
anim

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
 Position measured to be at  [1.23115578]
 Randomly measured state [9.] which has energy  0.5484827557301445
